# Laboration statistiska metoder

### Introduktion
I denna labb kommer jag att göra en multipel linjär regressions analys på ett dataset.

### Metod
Jag har skapat en regressionsklass i LinearRegression.py som hanterar alla min beräkningar. I denna klass har jag beräkningar som räknar ut: parametervektor, prediktioner, residualer, SSE, Syy, SSR, felvarians, RMSE, R2, kovariansmatris, t statistik, p värden, konfidensintervall, F statistik, F test p värde. Klassen innehåller även en summary() som printar ut resultatet för analys.

Jag kommer bygga 3 stycken modeller, en med alla numeriska features (model_full), en andra modell (model_ocean_full) med alla numeriska features samt en kategorisk featured som jag dummy kodat och en tredje modell (modell_reduced_ocean) som innehåller mindre features jämfört med första modellen men med samma kategoriska features tillagda som modell 2. Jag ska undersöka datasetet "housing.csv" som är ett dataset innehållande huspriser från California 1997.

1. model_full (alla numeriska variablar)
2. model_ocean_full (alla numeriska variablar + kategorisk dummy kodad variabel)
3. model_reduced_ocean (reducerade numeriska variablar + kategorisk dummy kodad variabel)

#### Databearbetning
Den enda databearbetning som gjort är exkluderandet av 207 NaN värden i kolumnen total_bedrooms. Ingen annan optimering har gjorts.

In [80]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from LinearRegression import LinearRegression

df = pd.read_csv("../Data/housing.csv")
df.describe()


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


De enda NaN värderna i datasetet finns i kolumnen: "total_bedrooms".

In [81]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

Jag droppar alla NaN värderna och får 207 mindre observationer att använda i min regressionanalys. Vi kan även här se de olika kategoriska värdena samt deras frekvens.

In [82]:
df_clean = df.dropna()
df.shape, df_clean.shape, df_clean["ocean_proximity"].value_counts()

((20640, 10),
 (20433, 10),
 ocean_proximity
 <1H OCEAN     9034
 INLAND        6496
 NEAR OCEAN    2628
 NEAR BAY      2270
 ISLAND           5
 Name: count, dtype: int64)

I regressionsanalysen valdes "median_house_value" som målvariablen (y) och de andra 8 numeriska kolumnerna som förklarande variablar i modell 1.

In [83]:
X_numeric = df_clean.iloc[:, :8].to_numpy()
y = df_clean.iloc[:, 8].to_numpy()
X_numeric.shape, y.shape

((20433, 8), (20433,))

### Features
I tabellen ovan ser vi att "total_rooms, "total_bedrooms, "population", och "households" har starka inbördes korrelation till varandra vilket indikerar att de alla visar samma överlappande information i datan.
Jag väljer därför att droppa "total_bedrooms och "households" och behåller "population" samt "total_rooms". Trots 86% kolinjäritet så behöll jag dessa 2 features för att de visar olika delar av bostadsmarknaden. "Population" är ett bra representativt mått för områdets efterfrågan, "total_rooms" beskriver områdets utbud.

Vi kan även se att "longitude" och "latitude" har en stark negativ korrelation med varandra. Det är ganska uppenbart då de är koordinater på öst-väst och nord-syd. Jag behåller dessa 2 för att dom är viktiga att kunna beskriva geografiska läget i modellen. Vi ska komma ihåg att inom fastighetsbranschen är det location, location, location som gäller.

In [84]:
model = LinearRegression()
model.fit(X_numeric, y)

pearson_corr = model.pearson()
feature_names = df_clean.columns[:8]
threshold = 0.8

[
    (feature_names[i], feature_names[j], pearson_corr[i, j])
    for i in range(len(feature_names))
    for j in range(i + 1, len(feature_names))
    if abs(pearson_corr[i, j]) > threshold
]

[('longitude', 'latitude', np.float64(-0.9246161131160107)),
 ('total_rooms', 'total_bedrooms', np.float64(0.9303795046865075)),
 ('total_rooms', 'population', np.float64(0.8572812510982971)),
 ('total_rooms', 'households', np.float64(0.9189915343453183)),
 ('total_bedrooms', 'population', np.float64(0.8777467431529307)),
 ('total_bedrooms', 'households', np.float64(0.9797282708045647)),
 ('population', 'households', np.float64(0.9071859001745))]

## Modell 1 - Alla numeriska features

Förklaringsgraden i modell 1 är 64% av variationen i huspriserna där RMSE eller genomsnittsfelet i modellen är ca $70 000$. Räknar man RMSE mot det genomsnittliga huspriset i datasetet ($206 000$) så är det 34% av snittet. Det visar att trots att modellen förklarar en del av variation så är prediktionsavvikelsen fortfarande relativt stor. Vidare så ser vi att modellens F-värde är 4478 och modellens P-värde är nära noll vilket visar att denna modell är statistiskt signifikant och att nollhypotesen kan förkastas. Samtliga variabler är statistiskt signifikanta enligt deras t och p värden, detta kan delvis förklaras av det stora antalet observationer i datasetet. Kollar vi på median_income så är t-värdet extremt höga 119, vilket visar på statistisk signifikans och är det starkaste förklaringsvariabeln i modellen. Resultatet är logiskt ekonomiskt då områden med högre inkomst tenderar att ha högre fastighetsvärden men värt att tänka på är att t-värdena påverkas av de många observationerna. Tittar man på latitude och longitude så är även dom starkt signifikanta vilket indikerar att geografiskt läge har stor betydelse för huspriset. Något som jag kommer analysera mer av i modell 2 när vi även lägger till kategorisk data om närhet till havet.



In [85]:
numeric_names = list(df_clean.columns[:8])

model_full = LinearRegression()
model_full.fit(X_numeric, y)
model_full.feature_names = ["Intercept"] + numeric_names

model_df, coef_df = model_full.summary()
display(model_df)
display(coef_df)

,n,d,df,R2,Sigma,Variance,RMSE,F-statistic,F p-value
0,20433,8,20424,0.6369,69571.4719,4.840190e+09,69556.1484,4478.3472,0.0


,Parameter,Beta,Std.error,t-värde,p-värde,CI nedre,CI övre
0,Intercept,-3.585396e+06,62900.5428,-57.0010,0.000000e+00,-3.708686e+06,-3.462106e+06
1,longitude,-4.273012e+04,717.0870,-59.5885,0.000000e+00,-4.413567e+04,-4.132457e+04
2,latitude,-4.250974e+04,676.9516,-62.7958,0.000000e+00,-4.383662e+04,-4.118286e+04
3,housing_median_age,1.157900e+03,43.3886,26.6867,2.946266e-154,1.072855e+03,1.242945e+03
4,total_rooms,-8.249700e+00,0.7943,-10.3867,3.294848e-25,-9.806500e+00,-6.692900e+00
5,total_bedrooms,1.138207e+02,6.9306,16.4229,3.188906e-60,1.002362e+02,1.274052e+02
6,population,-3.838560e+01,1.0841,-35.4071,1.459679e-266,-4.051050e+01,-3.626060e+01
7,households,4.770140e+01,7.5466,6.3209,2.653505e-10,3.290950e+01,6.249320e+01
8,median_income,4.029752e+04,337.2072,119.5038,0.000000e+00,3.963657e+04,4.095847e+04


## Modell 2 - Alla numeriska features + kategoriskt data

Modell 2 är byggd på alla numeriska features i dataset + den kategoriska datan som är "dummy-encodad" . I denna modell kan vi se att förklaringsgraden är förbättrad med ca 1% och att snittet på prediktionsfelet har minskat med $921$. Det tyder på att den kategoriska variabeln tillför förklaringskraft från modell 1. Modellens F- och P-värde visar att modellens fortfarande är signifikant men sjunker jämfört med tidigare då fler variabler (d) påverkar uträkningen. Om vi även jämför Latitude och Longitude så ser vi att t-värdet har gått ner då ocean_proximity kategorin minskar behovet av att koordinaterna visar lägeseffekten. En kategori (<1H ocean) har exkluderats vid "dummy-encoding" och används som referenskategori till de andra kategorierna. Även om kategorierna ej är jättespecifika så visar dom att ett hus Inland kostar i genomsnitt 39 000$ mindre, ett hus på en ö kostar i genomsnitt $152 000$ mer och att både near bay och near ocean visar mindre effekter men forfarande är statistiskt signifikanta. Modell 2 visar generellt sätt en bättre förklaringsgrad och mindre prediktionsfel samtidigt som närheten till havet har en signifikant påverkan på huspriserna.

In [86]:
ocean_dummies = pd.get_dummies(df_clean["ocean_proximity"], drop_first=True)
X_ocean_full = np.column_stack([X_numeric, ocean_dummies.to_numpy()])
ocean_dummies_names = list(ocean_dummies.columns)
feature_names_ocean_full = numeric_names + ocean_dummies_names

model_ocean_full = LinearRegression()
model_ocean_full.fit(X_ocean_full, y)
model_ocean_full.feature_names = ["Intercept"] + feature_names_ocean_full
model_df_ocean, coef_df_ocean = model_ocean_full.summary()
display(model_df_ocean)
display(coef_df_ocean)


,n,d,df,R2,Sigma,Variance,RMSE,F-statistic,F p-value
0,20433,12,20420,0.6465,68656.9511,4.713777e+09,68635.1069,3111.608,0.0


,Parameter,Beta,Std.error,t-värde,p-värde,CI nedre,CI övre
0,Intercept,-2.269954e+06,88013.8817,-25.7909,2.290199e-144,-2.442468e+06,-2.097440e+06
1,longitude,-2.681299e+04,1019.6506,-26.2963,6.596443e-150,-2.881159e+04,-2.481439e+04
2,latitude,-2.548218e+04,1004.7019,-25.3629,9.392264e-140,-2.745148e+04,-2.351289e+04
3,housing_median_age,1.072520e+03,43.8857,24.4389,4.854808e-130,9.865005e+02,1.158540e+03
4,total_rooms,-6.193300e+00,0.7915,-7.8250,5.321601e-15,-7.744600e+00,-4.641900e+00
5,total_bedrooms,1.005563e+02,6.8685,14.6402,2.734587e-48,8.709350e+01,1.140191e+02
6,population,-3.796910e+01,1.0761,-35.2824,9.349497e-265,-4.007840e+01,-3.585970e+01
7,households,4.961730e+01,7.4513,6.6589,2.829532e-11,3.501220e+01,6.422250e+01
8,median_income,3.925957e+04,338.0048,116.1510,0.000000e+00,3.859706e+04,3.992209e+04
9,INLAND,-3.928430e+04,1744.2578,-22.5221,5.662756e-111,-4.270319e+04,-3.586542e+04


 ## Modell 3 - Reducerade numeriska features + kategorisk data

Modell 3 är byggd på reducerat antalet features jämfört med tidigare modeller + tidigare kategorisk data. Jag ville testa ifall jag kunda förenkla modellen utan att förlora alltför mycket förklaringskraft. Features som exkluderades var total_bedrooms och households på grund av av starka korrelationer till total_rooms och population. Vi kan se att R2 är 62,5% och RMSE $70 691$ där både förklaringsgrad och prediktionsfel är sämre än tidigare. Modellen är fortfarande starkt signifikant med F-statistika på 3403, och P-värde ~0. Något intressant med denna modell är att total_rooms nu är positiv och stark signifikant koefficient jämfört med tidigare modeller. Det kan förklaras med att variablerna som togs bort hade en stark multikollinearitet med varandra, att dom konkurrerade om liknande information, något som också visades tidigare i pearsons korrelationsmatris. Sammanfattningsvis är denna modell lite förenklad med mindre variabler och eventuellt mer tolkningsbar men på bekostnad av R2 och RMSE.

In [87]:
keep_idx = [0,1,2,3,5,7]

X_reduced = X_numeric[:, keep_idx]
X_reduced_ocean = np.column_stack([X_reduced, ocean_dummies.to_numpy()])

reduced_numeric_names = [numeric_names[i] for i in keep_idx]
feature_names_reduced_ocean = reduced_numeric_names + ocean_dummies_names

model_reduced_ocean = LinearRegression()
model_reduced_ocean.fit(X_reduced_ocean, y)
model_reduced_ocean.feature_names = ["Intercept"] + feature_names_reduced_ocean

model_df_red, coef_df_red = model_reduced_ocean.summary()
display(model_df_red)
display(coef_df_red)


,n,d,df,R2,Sigma,Variance,RMSE,F-statistic,F p-value
0,20433,10,20422,0.625,70710.0204,4.999907e+09,70690.9846,3403.19,0.0


,Parameter,Beta,Std.error,t-värde,p-värde,CI nedre,CI övre
0,Intercept,-2.340426e+06,89653.3213,-26.1053,8.402224e-148,-2.516153e+06,-2.164698e+06
1,longitude,-2.811858e+04,1039.8383,-27.0413,2.942074e-158,-3.015674e+04,-2.608041e+04
2,latitude,-2.702130e+04,1027.7445,-26.2918,7.376042e-150,-2.903576e+04,-2.500684e+04
3,housing_median_age,1.003528e+03,45.1118,22.2453,2.420494e-108,9.151050e+02,1.091951e+03
4,total_rooms,1.522470e+01,0.4969,30.6387,1.344146e-201,1.425070e+01,1.619870e+01
5,population,-2.565420e+01,0.9327,-27.5044,1.475665e-163,-2.748240e+01,-2.382590e+01
6,median_income,3.362952e+04,301.0844,111.6947,0.000000e+00,3.303937e+04,3.421967e+04
7,INLAND,-4.688029e+04,1782.0504,-26.3069,5.021255e-150,-5.037325e+04,-4.338733e+04
8,ISLAND,1.526174e+05,31651.8751,4.8217,1.433302e-06,9.057714e+04,2.146576e+05
9,NEAR BAY,-1.389782e+03,1968.7678,-0.7059,4.802493e-01,-5.248725e+03,2.469161e+03


## Slutsats

Sammanfattningsvis visar analysen att alla tre modeller är statistiskt signifikanta och förklarar en del av variationen i huspriserna. Modell 1 hade en förklaringsgrad på 64% och en RMSE på 69 556$. Både förklaringsgraden ökar och prediktionsfelet minskar marginellt i modell 2 när kategorisk data om närhet till havet läggs till, vilket tyder på att läget är en viktigt faktor i prisbildningen. Den reducerade modell 3 är lite mer förenklad och har sämre förklaringsgrad och prediktionsfel än de 2 andra modellerna vilket är en avvägning om att förenkla modellen mot sämre prediktivitet. Konfidensintervallen är något smalare i modell 3 vilket indikerar mer stabila skattningar då vissa variabler exkluderats som haft starkt korrelation, vilket kan ha bidragit till multikollinearitet i de 2 första modellerna.